# Run **Expedited Internet Bypass Protocol** on FABRIC Nodes

## Input Required Information

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of the slice you wish to work on. |
| NODE_TO_FAIL | Node which will lose access to an interface. |
| INTF_TO_FAIL | Interface that will be failed on the node. |

In [48]:
SLICE_NAME = "eibp"

#change based on slice details
NODE_TO_FAIL = "a2"
INTF_TO_FAIL = "eth2"

## Access the Slice

The orchestrator class is initalized, which also means the slice and its nodes are now accessable as well.

In [49]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")

Slice name: eibp
Slice and nodes were acquired successfully.


## Delete the Log from a Prior Test if Necessary

In [50]:
rmLogCmd = "rm EIBP_*.log"
manager.executeCommandsParallel(rmLogCmd, prefixList="a,d,c")

Starting command on node c1
Command to execute: rm EIBP_*.log
Starting command on node d1
Command to execute: rm EIBP_*.log
Starting command on node d2
Command to execute: rm EIBP_*.log
Starting command on node a1
Command to execute: rm EIBP_*.log
Starting command on node a2
Command to execute: rm EIBP_*.log
Waiting for result from node c1
stdout: 
stderr: 
Waiting for result from node d1
stdout: 
stderr: 
Waiting for result from node d2
stdout: 
stderr: 
Waiting for result from node a1
stdout: 
stderr: 
Waiting for result from node a2
stdout: 
stderr: 


## EIBP **Initial Convergence**

Delay by a bit to get everything working first. The spines are started first, then 5 seconds later the leaves are.

In [51]:
import time

with open('config_small.txt','r') as file:
    config_data = {}
    
    for line in file:
        parts = line.strip().split(',')
        if len(parts) == 2:
            node_name, command = parts
            config_data[node_name] = command
            

    for node_name, command in config_data.items():
        print(f"Node:{node_name}, Command:{command}")
        
        startCmd = f"tmux new-session -d -s MNLR 'cd ~/MNLR_Code; {command}'"
        print(startCmd)
        
        manager.executeCommandsParallel(startCmd, prefixList = node_name)
        time.sleep(5)

Node:c1, Command:sudo ./MNLR -T 1 -L 1.1 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 1 -L 1.1 -N 1'
Starting command on node c1
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 1 -L 1.1 -N 1'
Waiting for result from node c1
stdout: 
stderr: 
Node:d1, Command:sudo ./MNLR -T 2 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Starting command on node d1
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Waiting for result from node d1
stdout: 
stderr: 
Node:d2, Command:sudo ./MNLR -T 2 -N 1
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Starting command on node d2
Command to execute: tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 2 -N 1'
Waiting for result from node d2
stdout: 
stderr: 
Node:a1, Command:sudo ./MNLR -T 3 -N 0 192.168.4.254 24 eth3
tmux new-session -d -s MNLR 'cd ~/MNLR_Code; sudo ./MNLR -T 3 -N 0 192.168.4.254 24 eth3'
Starting 

# wait for a minute 

# EIBP **Reconvergence** Testing

## Take the Interface down

This code brings down a network interface specified by {INTF_TO_FAIL} across multiple nodes and then retrieves IP addresses from the specified nodes in parallel using Python's manager.executeCommandsParallel function.

In [52]:
# Take the specified interface down
#intfName = f"{NODE_TO_FAIL}-{INTF_TO_FAIL}-p1"
#intf = manager.slice.get_interface(intfName)
#intf.ip_link_down()

downCmd = f"sudo ip link set down {INTF_TO_FAIL}"
manager.executeCommandsParallel(downCmd, prefixList=NODE_TO_FAIL)
downCmd = f"sudo ip addr"
manager.executeCommandsParallel(downCmd, prefixList=NODE_TO_FAIL)

Starting command on node a2
Command to execute: sudo ip link set down eth2
Waiting for result from node a2
stdout: 
stderr: 
Starting command on node a2
Command to execute: sudo ip addr
Waiting for result from node a2
stdout: 1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:d4:89:13 brd ff:ff:ff:ff:ff:ff
    altname enp3s0
    inet 10.30.6.225/23 brd 10.30.7.255 scope global dynamic noprefixroute eth0
       valid_lft 48264sec preferred_lft 48264sec
    inet6 2001:400:a100:3030:f816:3eff:fed4:8913/64 scope global dynamic noprefixroute 
       valid_lft 86396sec preferred_lft 14396sec
    inet6 fe80::f816:3eff

# wait for a minute

## Stop EIBP on Each Node
# let us kill from a1, a2, d1, d2 and c1

In [53]:
stopCmd = "tmux kill-session -t MNLR"
manager.executeCommandsParallel(stopCmd, prefixList="a")
manager.executeCommandsParallel(stopCmd, prefixList="d")
manager.executeCommandsParallel(stopCmd, prefixList="c")

Starting command on node a1
Command to execute: tmux kill-session -t MNLR
Starting command on node a2
Command to execute: tmux kill-session -t MNLR
Waiting for result from node a1
stdout: 
stderr: 
Waiting for result from node a2
stdout: 
stderr: 
Starting command on node d1
Command to execute: tmux kill-session -t MNLR
Starting command on node d2
Command to execute: tmux kill-session -t MNLR
Waiting for result from node d1
stdout: 
stderr: 
Waiting for result from node d2
stdout: 
stderr: 
Starting command on node c1
Command to execute: tmux kill-session -t MNLR
Waiting for result from node c1
stdout: 
stderr: 


## Collect Log Results

Now that the nodes have logged updates to their respective log files, they need to be downloaded to be analyzed.

In [54]:
# View contents on each node

# compileCmd = "ls"
# manager.executeCommandsParallel(compileCmd, prefixList="a,d,c")

In [55]:
import os

LOG_DIR_PATH = "../logs"
LOG_NAME = "EIBP_{name}.log"
logPath = os.path.join(LOG_DIR_PATH, LOG_NAME)

# If the logs directory does not already exist, create it
if not os.path.exists(LOG_DIR_PATH):
    os.makedirs(LOG_DIR_PATH)
    
manager.downloadFilesParallel(logPath, LOG_NAME, prefixList="a,d,c", addNodeName=True)

Starting download on node c1
File to download: EIBP_c1.log
Location of download: ../logs/EIBP_c1.log
Starting download on node d1
File to download: EIBP_d1.log
Location of download: ../logs/EIBP_d1.log
Starting download on node d2
File to download: EIBP_d2.log
Location of download: ../logs/EIBP_d2.log
Starting download on node a1
File to download: EIBP_a1.log
Location of download: ../logs/EIBP_a1.log
Starting download on node a2
File to download: EIBP_a2.log
Location of download: ../logs/EIBP_a2.log
Waiting for result from node c1
Output: None
Waiting for result from node d1
Output: None
Waiting for result from node d2
Output: None
Waiting for result from node a1
Output: None
Waiting for result from node a2
Output: None


## Bring the Interface Back Up

In [56]:
#intf.ip_link_up()
upCmd = f"sudo ip link set up {INTF_TO_FAIL}"
manager.executeCommandsParallel(upCmd, prefixList=NODE_TO_FAIL)
upCmd = f"sudo ip addr"
manager.executeCommandsParallel(upCmd, prefixList=NODE_TO_FAIL)

Starting command on node a2
Command to execute: sudo ip link set up eth2
Waiting for result from node a2
stdout: 
stderr: 
Starting command on node a2
Command to execute: sudo ip addr
Waiting for result from node a2
stdout: 1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 9000 qdisc fq_codel state UP group default qlen 1000
    link/ether fa:16:3e:d4:89:13 brd ff:ff:ff:ff:ff:ff
    altname enp3s0
    inet 10.30.6.225/23 brd 10.30.7.255 scope global dynamic noprefixroute eth0
       valid_lft 48176sec preferred_lft 48176sec
    inet6 2001:400:a100:3030:f816:3eff:fed4:8913/64 scope global dynamic noprefixroute 
       valid_lft 86309sec preferred_lft 14309sec
    inet6 fe80::f816:3eff:f